In [ ]:
# Installer nødvendige pakker
# !pip install pymupdf4llm pymupdf requests

In [2]:
import requests
import pymupdf4llm
import fitz
import re
import json
from pathlib import Path
from datetime import datetime

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


In [6]:
def clean_markdown(md: str) -> str:
    """Strip page numbers and clean spacing"""
    md = re.sub(r"(?m)^\s*\d+\s*$", "", md)  # remove number-only lines
    md = re.sub(r"\n{3,}", "\n\n", md)
    return md.strip()
    
def extract_pdf_to_jsonl(
    pdf_path: Path,
    out_dir: Path,
    source: str,
    domain: str,
    start_page: int = 0,
    end_page: int | None = None,
    source_type: str = "pdf",
) -> Path:
    """
    Konverter lokal PDF til markdown → lagre som JSONL med metadata.
    Returnerer sti til .jsonl-fila.
    """
    out_dir.mkdir(parents=True, exist_ok=True)

    # Filnavn basert på tittel
    slug = source.lower().replace(" ", "-")
    out_jsonl = out_dir / f"{slug}.jsonl"

    # Finn antall sider
    with fitz.open(pdf_path) as doc:
        if end_page is None:
            end_page = doc.page_count
        print(f"📄 {doc.page_count} sider (leser {start_page+1}–{end_page})")

    # PDF → Markdown
    md_text = pymupdf4llm.to_markdown(str(pdf_path), pages=range(start_page, end_page))
    md_text = clean_markdown(md_text)

    # Lag JSON-record
    record = {
        "source": source,
        "domain": domain,
        "source_url": str(pdf_path),
        "source_type": source_type,
        "text_format": "markdown",
        "text": md_text,
        "retrieved_at": datetime.utcnow().isoformat(timespec="seconds") + "Z",
    }

    # Skriv utfil
    out_jsonl.write_text(json.dumps(record, ensure_ascii=False) + "\n", encoding="utf-8")
    print(f"💾 Skrev {out_jsonl}")
    return out_jsonl

## Kjør konvertering av Statsbudsjett PDF

Oppdater URL og metadata under for å laste ned og konvertere statsbudsjett.pdf

In [7]:
# Konfigurasjon
PDF_PATH = Path("../../data/statsbudsjett.pdf")
OUTPUT_DIR = Path("./output")
SOURCE_NAME = "Statsbudsjett 2025"
DOMAIN = "offentlig"

# Kjør konvertering
output_file = extract_pdf_to_jsonl(
    pdf_path=PDF_PATH,
    out_dir=OUTPUT_DIR,
    source=SOURCE_NAME,
    domain=DOMAIN,
    start_page=0,
    end_page=None,  # None = alle sider
)

print(f"\n✅ Ferdig! Output: {output_file}")

📄 190 sider (leser 1–190)
💾 Skrev output/statsbudsjett-2025.jsonl

✅ Ferdig! Output: output/statsbudsjett-2025.jsonl
💾 Skrev output/statsbudsjett-2025.jsonl

✅ Ferdig! Output: output/statsbudsjett-2025.jsonl


/var/folders/3f/zfgffxv133jfcvdrwt2l6qcm0000gn/T/ipykernel_30970/922006593.py:44: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "retrieved_at": datetime.utcnow().isoformat(timespec="seconds") + "Z",


## Les og inspiser resultat

In [8]:
# Les JSONL-fila
with open(output_file, "r", encoding="utf-8") as f:
    data = json.loads(f.read())

print(f"Source: {data['source']}")
print(f"Domain: {data['domain']}")
print(f"Text length: {len(data['text'])} chars")
print(f"\nFirst 500 chars:\n{data['text'][:500]}...")

Source: Statsbudsjett 2025
Domain: offentlig
Text length: 546690 chars

First 500 chars:
(Korrigert utgave per 14.11.2025)

# **Meld. St. 1**

###### (2025 – 2026) Melding til Stortinget

### Nasjonalbudsjettet 2026

# **Meld. St. 1**

###### (2025–2026) Melding til Stortinget

### Nasjonalbudsjettet 2026

##### **Innhold**

**1** **Hovedlinjer i den økonomiske**
**politikken og utsiktene**
**for norsk økonomi** ...................... 5

**2** **De økonomiske utsiktene** ......... 12
2.1 Internasjonal konjunktursituasjon .......................................... 12
2.2 Norsk konjun...


In [ ]:
# Lagre markdown direkte til fil
md_output = OUTPUT_DIR / f"{SOURCE_NAME.lower().replace(' ', '-')}.md"
md_output.write_text(data['text'], encoding='utf-8')
print(f"✅ Markdown lagret til: {md_output}")
print(f"📊 Størrelse: {len(data['text']):,} tegn")

## Lagre som Markdown fil